<a href="https://colab.research.google.com/github/wtvieira/Projeto_Situacional_Educacao/blob/main/Tratamento_Dados_Rede_Municipal_Recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pyspark
# importar SparkSession
from pyspark.sql import SparkSession

# criar SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# checar SparkSession
spark

In [8]:
#Conexão com os arquivos base no Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# ler base de dados
#Base de Informações das Escolas
df_Info_Escolas_Original = spark.read.csv("/content/drive/MyDrive/Projeto_Situacional_Educacao/info_escolas_2023_27122023.csv",
                    header=True,
                    inferSchema = True,
                    sep = ";")
df_Info_Escolas_Original.show(5)
#Base de Informações dos Alunos
df_Info_Alunos_Original = spark.read.csv("/content/drive/MyDrive/Projeto_Situacional_Educacao/situacaofinalalunos2023_22122023.csv",
                    header=True,
                    inferSchema = True,
                    sep = ";")
df_Info_Alunos_Original.show(5)

+---+--------+----------------+----------+--------------------+--------+------------------+------+----------+------------------+--------+----------+----------+---------------+------------------+-----------+--------------+-----------------+----------+------------+--------------------+---------+--------+
|rpa|tipo_cod|            tipo|cod_escola|              escola|    inep|               rua|numero|cod_bairro|            bairro|metragem|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|data_visita|quadra_coberta|quadra_descoberta|biblioteca|sala_recurso|              gestor|longitude|latitude|
+---+--------+----------------+----------+--------------------+--------+------------------+------+----------+------------------+--------+----------+----------+---------------+------------------+-----------+--------------+-----------------+----------+------------+--------------------+---------+--------+
|  1|       1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|26120658|         CAMUTAN

In [37]:
df_Info_Escolas_Original.printSchema()

root
 |-- rpa: integer (nullable = true)
 |-- tipo_cod: integer (nullable = true)
 |-- tipo: string (nullable = true)
 |-- cod_escola: integer (nullable = true)
 |-- escola: string (nullable = true)
 |-- inep: integer (nullable = true)
 |-- rua: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- cod_bairro: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- metragem: string (nullable = true)
 |-- qtd_alunos: integer (nullable = true)
 |-- qtd_turmas: integer (nullable = true)
 |-- qtd_professores: integer (nullable = true)
 |-- escola_climatizada: string (nullable = true)
 |-- data_visita: string (nullable = true)
 |-- quadra_coberta: string (nullable = true)
 |-- quadra_descoberta: string (nullable = true)
 |-- biblioteca: string (nullable = true)
 |-- sala_recurso: string (nullable = true)
 |-- gestor: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)



In [38]:
#LIMPANDO A BASE DE DADOS base_Info_Escolas_Original
df_Info_Escolas = df_Info_Escolas_Original.select("rpa", "tipo_cod", "tipo", "cod_escola", "escola", "inep", "qtd_alunos", "qtd_turmas", "qtd_professores", "longitude", "latitude")
df_Info_Escolas.show(5)

+---+--------+----------------+----------+--------------------+--------+----------+----------+---------------+---------+--------+
|rpa|tipo_cod|            tipo|cod_escola|              escola|    inep|qtd_alunos|qtd_turmas|qtd_professores|longitude|latitude|
+---+--------+----------------+----------+--------------------+--------+----------+----------+---------------+---------+--------+
|  1|       1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|26120658|       429|        18|             13|-34.89535|-8.07895|
|  1|       1|ESCOLA MUNICIPAL|       289|CIDADAO HERBERT D...|26121972|       163|        12|             11|-34.88707|-8.04576|
|  1|       1|ESCOLA MUNICIPAL|       256|            DO COQUE|26122898|       622|        26|             18|-34.90022|-8.07145|
|  1|       1|ESCOLA MUNICIPAL|        91|         DOS COELHOS|26118670|       730|        35|             24|-34.89097|-8.06876|
|  1|       1|ESCOLA MUNICIPAL|       384|   FREI TADEU GLASER|26175347|       130|       

In [41]:
#SINCRONIZANDO O NOME DA COLUNA COM
df_Info_Escolas = df_Info_Escolas.withColumnRenamed("escola","NESCOLNOME")
df_Info_Escolas.show(5)

+---+--------+----------------+----------+--------------------+--------+----------+----------+---------------+---------+--------+
|rpa|tipo_cod|            tipo|cod_escola|          NESCOLNOME|    inep|qtd_alunos|qtd_turmas|qtd_professores|longitude|latitude|
+---+--------+----------------+----------+--------------------+--------+----------+----------+---------------+---------+--------+
|  1|       1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|26120658|       429|        18|             13|-34.89535|-8.07895|
|  1|       1|ESCOLA MUNICIPAL|       289|CIDADAO HERBERT D...|26121972|       163|        12|             11|-34.88707|-8.04576|
|  1|       1|ESCOLA MUNICIPAL|       256|            DO COQUE|26122898|       622|        26|             18|-34.90022|-8.07145|
|  1|       1|ESCOLA MUNICIPAL|        91|         DOS COELHOS|26118670|       730|        35|             24|-34.89097|-8.06876|
|  1|       1|ESCOLA MUNICIPAL|       384|   FREI TADEU GLASER|26175347|       130|       

In [44]:
#VERIFICAÇÃO DE LIGAÇÃO ENTRE AS DUAS BASES

#IMPORTAÇÃO DE BIBLIOTECA PARA UTILIZAÇÃO DE FUNÇÕES SQL
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#CRIAÇÃO DE DATAFRAME COM DADOS VERIFICADOS
df_verificacao = df_Info_Alunos.join(df_Info_Escolas, df_Info_Alunos.NESCOLNOME == df_Info_Escolas.NESCOLNOME, "inner")
df_verificacao.show(5)

+----+------------+----------+------------+----------+----------+------+-------+---------+----------+------------------+-----+----------+--------+----------+----------+-----+----+--------------------+---+--------+----------------+----------+------------+--------+----------+----------+---------------+---------+--------+
| ANO|  NESCOLNOME|EBAIRRNOME|  EESCOLRUAE|AESCOLNUME|CESCOLRPAE|MODENS|COMPENS|ANOENSINO|ESERMODESC|        NMODENNOME|TURMA|NTURESNOME|     MAT|CALUNOSEXO|DALUNONASC|IDADE|SITU|          ESITALDESC|rpa|tipo_cod|            tipo|cod_escola|  NESCOLNOME|    inep|qtd_alunos|qtd_turmas|qtd_professores|longitude|latitude|
+----+------------+----------+------------+----------+----------+------+-------+---------+----------+------------------+-----+----------+--------+----------+----------+-----+----+--------------------+---+--------+----------------+----------+------------+--------+----------+----------+---------------+---------+--------+
|2023|ABILIO GOMES|BOA VIAGEM|BRUNO V